In [2]:
import torch
from torchsummary import summary
import torch.nn as nn

In [10]:
class Discriminator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.activation = nn.LeakyReLU(0.05)
        self.dropout = nn.Dropout2d(0.2)
        self.initial_conv = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=7,padding=3,stride=1,bias=False),#no change
            nn.BatchNorm2d(num_features=64),
            nn.LeakyReLU(0.05),
            nn.Conv2d(64,64,kernel_size=7,padding=3,stride=2,bias=False), # /2
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.05)
        )
        self.conv1 = self._normal_block(kernel=1,padding=0,stride=1,
                                        in_c=64,out_c=256)

        self.bottleneck1 = self._bottleneck_block(kernel=[1,3,1],
                                                  padding=[0,1,0],
                                                  stride=[1,1,1],
                                                  in_c=64,out_c=256,middle_c=64)
        self.bottleneck2 = self._bottleneck_block(kernel=[1,3,1],
                                                  padding=[0,1,0],
                                                  stride=[1,1,1],
                                                  in_c=256,out_c=256,middle_c=64)
        self.conv3 = self._normal_block(kernel=1,padding=0,stride=2,
                                        in_c=256,out_c=512)
        self.bottleneck3 = self._bottleneck_block(kernel=[1,3,1],
                                                  padding=[0,1,0],
                                                  stride=[1,2,1],
                                                  in_c=256,out_c=512,middle_c=128)


    def _normal_block(self,kernel,padding,stride,in_c,out_c,):
        block = nn.Sequential(
            nn.Conv2d(in_c,out_c,
                      kernel_size=kernel,padding=padding,stride=stride,bias=False),
            nn.BatchNorm2d(out_c)
        )
        return block
    
    def _bottleneck_block(self,kernel,padding,stride,in_c,out_c,middle_c):
        block = nn.Sequential(
            nn.Conv2d(in_c,middle_c,kernel_size=kernel[0],
                      padding = padding[0],stride=stride[0],bias=False),
            nn.BatchNorm2d(middle_c),
            nn.LeakyReLU(0.05),
            nn.Conv2d(middle_c,middle_c,kernel_size=kernel[1],
                      padding = padding[1],stride=stride[1],bias=False),
            nn.BatchNorm2d(middle_c),
            nn.LeakyReLU(0.05),
            nn.Conv2d(middle_c,out_c,kernel_size=kernel[2],
                      padding = padding[2],stride=stride[2],bias=False),
            nn.BatchNorm2d(out_c),
        )
        return block
    
    def forward(self, x):
        output = self.initial_conv(x)
        output = self.dropout(output)
        resnet_1 = self.activation(self.conv1(output) + self.bottleneck1(output))
        resnet_1 = self.dropout(resnet_1)
        resnet_2 = self.activation(resnet_1 + self.bottleneck2(resnet_1))
        resnet_2 = self.dropout(resnet_2)
        resnet_3 = self.activation(self.conv3(resnet_2) + self.bottleneck3(resnet_2))
        resnet_3 = self.dropout(resnet_3)
        return resnet_3

In [11]:
discriminator = Discriminator()
summary(discriminator,torch.zeros(1,3,128,128))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 64, 64]          --
|    └─Conv2d: 2-1                       [-1, 64, 128, 128]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 128, 128]        128
|    └─LeakyReLU: 2-3                    [-1, 64, 128, 128]        --
|    └─Conv2d: 2-4                       [-1, 64, 64, 64]          200,704
|    └─BatchNorm2d: 2-5                  [-1, 64, 64, 64]          128
|    └─LeakyReLU: 2-6                    [-1, 64, 64, 64]          --
├─Dropout2d: 1-2                         [-1, 64, 64, 64]          --
├─Sequential: 1-3                        [-1, 256, 64, 64]         --
|    └─Conv2d: 2-7                       [-1, 256, 64, 64]         16,384
|    └─BatchNorm2d: 2-8                  [-1, 256, 64, 64]         512
├─Sequential: 1-4                        [-1, 256, 64, 64]         --
|    └─Conv2d: 2-9                       [-1, 64, 64, 64]          4,0

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 64, 64]          --
|    └─Conv2d: 2-1                       [-1, 64, 128, 128]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 128, 128]        128
|    └─LeakyReLU: 2-3                    [-1, 64, 128, 128]        --
|    └─Conv2d: 2-4                       [-1, 64, 64, 64]          200,704
|    └─BatchNorm2d: 2-5                  [-1, 64, 64, 64]          128
|    └─LeakyReLU: 2-6                    [-1, 64, 64, 64]          --
├─Dropout2d: 1-2                         [-1, 64, 64, 64]          --
├─Sequential: 1-3                        [-1, 256, 64, 64]         --
|    └─Conv2d: 2-7                       [-1, 256, 64, 64]         16,384
|    └─BatchNorm2d: 2-8                  [-1, 256, 64, 64]         512
├─Sequential: 1-4                        [-1, 256, 64, 64]         --
|    └─Conv2d: 2-9                       [-1, 64, 64, 64]          4,0

In [9]:
discriminator

Discriminator(
  (activation): LeakyReLU(negative_slope=0.05)
  (initial_conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.05)
    (3): Conv2d(64, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.05)
  )
  (conv1): Sequential(
    (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (bottleneck1): Sequential(
    (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.05)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 